In [1]:
import torch
import time
import os
from tempfile import TemporaryDirectory
from tqdm import tqdm
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn as nn
from torchvision import datasets, models, transforms
device = "cuda"

f:\Users\mario\.conda\envs\parking\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [33]:
pk_lot_dir  = "pk_lot_data"
cnr_parking_dir  = "cnr_parking_data"

data_transforms = {
        "train": transforms.Compose(
            [
                transforms.RandomResizedCrop(224),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            ]
        ),
        "val": transforms.Compose(
            [
                transforms.Resize(256),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            ]
        ),
    }

image_datasets_pk_lot = {
    x: datasets.ImageFolder(os.path.join(pk_lot_dir, y), data_transforms[y])
    for x,y  in zip(["train_pk", "val_pk"], ["train", "val"])
}
image_datasets_cnr_park = {
    x: datasets.ImageFolder(os.path.join(cnr_parking_dir, y), data_transforms[y])
    for x,y  in zip(["train_cnr", "val_cnr"], ["train", "val"])
}



In [34]:
train_dataset = torch.utils.data.ConcatDataset([image_datasets_pk_lot["train_pk"], image_datasets_cnr_park["train_cnr"]])
val_dataset = torch.utils.data.ConcatDataset([image_datasets_pk_lot["val_pk"], image_datasets_cnr_park["val_cnr"]])

In [37]:
dataloaders= {
    "train": torch.utils.data.DataLoader(
        train_dataset,
        batch_size=32,
        shuffle=True,
        num_workers=0,
    ),
    "val": torch.utils.data.DataLoader(
        val_dataset,
        batch_size=32,
        shuffle=True,
        num_workers=0,
    )
    
}



In [38]:
dataloaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x24749b02950>,
 'val': <torch.utils.data.dataloader.DataLoader at 0x2475cf51310>}

In [39]:
dataset_sizes = {"train": len(train_dataset) , "val": len(val_dataset)}
print("Size of train and test: ", dataset_sizes)

Size of train and test:  {'train': 618925, 'val': 121009}


In [40]:
from utils.image_utils import mAlexNet

model = mAlexNet()
model.to(device)
model.load_state_dict(torch.load("models\m_alex_net.pth"))

<All keys matched successfully>

In [42]:
inputs, labels  = next(iter(dataloaders["val"]))

inputs = inputs.to(device)
labels = labels.to(device)
running_corrects=0
with torch.no_grad():
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        running_corrects += torch.sum(preds == labels.data)
        

accuracy = running_corrects.to(torch.float32) / dataset_sizes["val"]
accuracy

tensor(0.0003, device='cuda:0')

In [43]:
running_corrects = 0
for inputs, labels in tqdm(dataloaders["train"]):
    inputs = inputs.to(device)
    labels = labels.to(device)
    with torch.no_grad():
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        running_corrects += torch.sum(preds == labels.data)

        
        
train_accuracy = running_corrects.to(torch.float32) / dataset_sizes["train"]
train_accuracy

  3%|▎         | 623/19342 [03:52<2:24:08,  2.16it/s] 